In [1]:
import pandas as pd
import ast

qos_data_df = pd.read_csv("raw_data/qos_data.csv")
qos_curves_df = pd.read_csv("raw_data/qos_curves.csv")
qos_curves_df['X'] = qos_curves_df['X'].apply(ast.literal_eval)
qos_curves_df['Y'] = qos_curves_df['Y'].apply(ast.literal_eval)

In [2]:
merged_consumption_df = pd.merge(qos_data_df, qos_curves_df, left_on="CONSUMPTION_PROFILE_CURVE_ID", right_on="CURVE_ID")
merged_inventory_df = pd.merge(qos_data_df, qos_curves_df, left_on="INVENTORY_CURVE_ID", right_on="CURVE_ID")
len(merged_consumption_df), len(merged_inventory_df)

(12754, 12754)

In [3]:
all_locations = qos_data_df['LOCATION'].unique()
len(all_locations)

100

In [4]:
all_weeks = qos_curves_df['WEEK_START'].unique()
len(all_weeks)

2

In [5]:
from datetime import timedelta, datetime

def generate_datetime_range(start, end, num_points):
    total_seconds = int((end - start).total_seconds())
    delta_seconds = total_seconds / (num_points - 1)
    return [start + timedelta(seconds=i * delta_seconds) for i in range(num_points)]

def minutes_to_datetimes(minutes, week_start):
    start_date = datetime.strptime(week_start, '%d.%m.%Y')
    return [start_date + timedelta(minutes=int(minute)) for minute in minutes]

In [7]:
from collections import defaultdict

from scipy.interpolate import interp1d
import numpy as np

MINUTES_IN_A_WEEK = 7 * 24 * 60

location_qos_metric = defaultdict(dict)

for location in all_locations:
    for week in all_weeks:
        filtered_inventory_df = merged_inventory_df[(merged_inventory_df["LOCATION"] == location) & (merged_inventory_df["WEEK_START"] == week)]
        filtered_consumption_df = merged_consumption_df[(merged_consumption_df["LOCATION"] == location) & (merged_consumption_df["WEEK_START"] == week)]
        unique_products_df = filtered_inventory_df.drop_duplicates(subset=['PRODUCT'])
        unique_consumption_df = filtered_consumption_df.drop_duplicates(subset=['PRODUCT'])

        if unique_products_df.empty:
            location_qos_metric[location][week] = None
            continue
        
        availability_counts = {}
        total_product_counts = {}

        for i, row in unique_products_df.iterrows():
            converted_x_inventory = minutes_to_datetimes(row["X"], row["WEEK_START"])
            x_range_inventory = generate_datetime_range(converted_x_inventory[0], converted_x_inventory[-1], num_points=MINUTES_IN_A_WEEK)
            f_inventory = interp1d([dt.timestamp() for dt in converted_x_inventory], row["Y"], kind='linear')
            inventory_levels = f_inventory([dt.timestamp() for dt in x_range_inventory])

            for j, time_instant in enumerate(x_range_inventory):
                timestamp = time_instant.timestamp()
                inventory_level = inventory_levels[j]
        
                if timestamp not in availability_counts:
                    availability_counts[timestamp] = 0
                    total_product_counts[timestamp] = 0
        
                if inventory_level > 0:
                    # Product is available
                    availability_counts[timestamp] += 1
        
                total_product_counts[timestamp] += 1
        
        # Calculate the availability ratio
        availability_ratio = {
            timestamp: availability_counts[timestamp] / total_product_counts[timestamp] 
                              for timestamp in availability_counts
        }
        availability_ratio_list = sorted(availability_ratio.items())
        timestamps, location_availability_ratio = zip(*availability_ratio_list)

        converted_x_consumption = minutes_to_datetimes(unique_consumption_df['X'].iloc[0], unique_consumption_df["WEEK_START"].iloc[0])
        x_range_consumption = generate_datetime_range(converted_x_consumption[0], converted_x_consumption[-1], num_points=MINUTES_IN_A_WEEK)
        f_consumption = interp1d([dt.timestamp() for dt in converted_x_consumption], unique_consumption_df['X'].iloc[0], kind='linear')
        consumption_profile_interpolated = f_consumption([dt.timestamp() for dt in x_range_consumption])

        consumption_profile_cumulative = np.cumsum(consumption_profile_interpolated) / np.sum(consumption_profile_interpolated)
        availability_interp_func = interp1d(timestamps, list(availability_ratio.values()), bounds_error=False, fill_value="extrapolate")
        adjusted_availability_data = availability_interp_func(timestamps)

        area_under_curve = np.trapz(adjusted_availability_data, consumption_profile_cumulative)
        
        print(area_under_curve)
        location_qos_metric[location][week] = area_under_curve

0.4252991795908999
0.35162682163004616
0.31902609983582797
0.6496879163984035
0.21245866797322793
0.5827646727934455
0.5726897534728174
0.2711996001787896
0.39228848708955866
0.5008179741812171
0.5858021333843588
0.4815441215456098
0.6271578752243997
0.6001048695822483
0.6048655453268387
0.5796732106044786
0.6083344581056408
0.3824397845047215
0.45031171762556327
0.607709514210035
0.6598479633251809
0.48636971463795803
0.6126471772468399
0.4032780337581781
0.5842222194224708
0.606703454744555
0.42971683781101877
0.4568165774293615
0.41005247129924677
0.3663763895098849
0.3655960149666122
0.3973586883318751
0.7306007601002145
0.7325837846822072
0.788142369365705
0.6907160023118948
0.4440596926774871
0.604987407740037
0.5770603511122164
0.5225408397040008
0.6395217205075574
0.5581641396603199
0.5342638036853733
0.7463302563070645
0.5463980142194127
0.6211064058881923
0.4576845356746637
0.4754746914956261
0.33903520861242364
0.38055552781654084
0.6422512744556103
0.5105439941131726
0.3669

In [8]:
pd.DataFrame(location_qos_metric).to_csv('location_qos_metric.csv')